In [1]:
# Library imports
import pyforest
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
from tqdm import tqdm
from pprint import pprint
from time import sleep
import time
import seaborn as sns

from turtle import forward
import torch.nn as nn
import torch.nn.functional as F
import math
import torch
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
from torch.utils.data import DataLoader, TensorDataset

#### **Hyperparams and data-loading**

In [5]:
train_edges = np.load('movielens/train_edges.npy')
user_list = train_edges[:, 0]
item_list = train_edges[:, 1]
rating_list = train_edges[:, 2].astype('float32')

n_users = user_list.max() + 1 
n_items = item_list.max() + 1
n_samples = len(rating_list)

n_users, n_items, n_samples

(943, 1682, 200000)

#### **Neural collaborative filtering**

In [19]:
class NCF(Module):
    def __init__(self, n_users, n_items, n_factors):
        super(NCF, self).__init__()
        self.user_emb = nn.Embedding(n_users, n_factors)
        self.item_emb = nn.Embedding(n_items, n_factors)
        self.fc = nn.Linear(n_factors * 2, 1)
        # self.user_emb.weight.data.uniform_(0, 0.5)
        # self.item_emb.weight.data.uniform_(0, 0.5)
        # self.fc.weight.data.uniform_(0, 0.5)

    def forward(self, user, item):
        u = self.user_emb(user)
        i = self.item_emb(item)
        features = torch.concat([u, i], dim = 1)
        x = self.fc(features)
        out = torch.sigmoid(x)
        return out

In [25]:
class NCF(Module):
    def __init__(self, n_users, n_items, n_factors):
        super(NCF, self).__init__()
        self.user_emb = nn.Embedding(n_users, n_factors)
        self.item_emb = nn.Embedding(n_items, n_factors)
        self.fc1 = nn.Linear(n_factors * 2, n_factors)
        self.fc2 = nn.Linear(n_factors, 1)

    def forward(self, user, item):
        u = self.user_emb(user)
        i = self.item_emb(item)
        features = torch.concat([u, i], dim = 1)
        x = self.fc1(features)
        x = self.fc2(x)
        out = torch.sigmoid(x)
        return out

In [22]:
class CollaborativeFiltering(Module):
    def __init__(self, n_users, n_items, n_factors):
        super(CollaborativeFiltering, self).__init__()
        self.user_emb = nn.Embedding(n_users, n_factors)
        self.item_emb = nn.Embedding(n_items, n_factors)

    def forward(self, user, item):
        u = self.user_emb(user)
        i = self.item_emb(item)
        dot = (u * i).sum(1)
        return torch.sigmoid(dot)

In [4]:
def get_accuracy(y_hat, y):
    y = y.clone().int()
    y_hat = (y_hat.clone() > 0.5).int()
    accuracy = (y == y_hat).sum() / len(y)
    return accuracy.item()

#### **Test NCF**

In [31]:
# use gpu 
use_gpu = 6
if use_gpu == -1:
    device = 'cpu'
else:
    device = torch.device('cuda:{}'.format(str(use_gpu)) if torch.cuda.is_available() else 'cpu')

# load users, items and ratings as tensors
users = torch.tensor(user_list, device = device)
items = torch.tensor(item_list, device = device)
ratings = torch.tensor(rating_list, device = device)
ratings = ratings.reshape((len(ratings), 1))

# define model and it's parameters
n_factors = 64
lr = 100
T = 301
model = NCF(n_users, n_items, n_factors)
# model = CollaborativeFiltering(n_users, n_items, n_factors)
model.to(device)

manual_gradients = False
if manual_gradients == True:
    p1, p2 = model.parameters()
else:
    optimizer = torch.optim.SGD(model.parameters(), lr = 100)

torch.manual_seed(0)
for layer in model.children():
    layer.reset_parameters()

loss_fn = nn.BCELoss(reduction = 'mean')
model.train()

if manual_gradients == True:
    for i in range(T):
        y_hat = model(users, items)
        loss = loss_fn(y_hat, ratings)
        # results.append([delta, i, loss.item()])
        if i % 10 == 0:
            print('T = ', T, ' loss: ', loss.item(), ' acc: ', get_accuracy(y_hat, ratings))

        # compute inner parameter gradients
        p1_grad = torch.autograd.grad(loss, p1, retain_graph=retain_graph, create_graph=create_graph)
        p2_grad = torch.autograd.grad(loss, p2, retain_graph=retain_graph, create_graph=create_graph)

        # update inner parameters
        with torch.no_grad():
            p1_new = p1 - lr * p1_grad[0]
            p2_new = p2 - lr * p2_grad[0]
            p1.copy_(p1_new)
            p2.copy_(p2_new)
else:
    for i in range(T):
        y_hat = model(users, items)
        loss = loss_fn(y_hat, ratings)
        # results.append([delta, i, loss.item()])
        if i % 10 == 0:
            print('T = ', T, ' loss: ', loss.item(), ' acc: ', get_accuracy(y_hat, ratings))
            

        # use torch.optim optimizer to compute gradients
        optimizer.zero_grad()
        loss.backward(retain_graph=True, create_graph=False)
        optimizer.step()
    

T =  301  loss:  0.7098179459571838  acc:  0.4984299838542938
T =  301  loss:  32.774559020996094  acc:  0.6716349720954895
T =  301  loss:  30.654199600219727  acc:  0.6933099627494812
T =  301  loss:  30.435564041137695  acc:  0.6954999566078186
T =  301  loss:  30.250099182128906  acc:  0.697439968585968
T =  301  loss:  29.816503524780273  acc:  0.701770007610321
T =  301  loss:  29.847911834716797  acc:  0.7014899849891663
T =  301  loss:  29.60127830505371  acc:  0.7039799690246582
T =  301  loss:  29.617719650268555  acc:  0.7037999629974365
T =  301  loss:  29.589561462402344  acc:  0.7040449976921082
T =  301  loss:  29.512977600097656  acc:  0.7048049569129944
T =  301  loss:  29.384414672851562  acc:  0.7060949802398682
T =  301  loss:  29.336135864257812  acc:  0.7066049575805664
T =  301  loss:  29.172136306762695  acc:  0.7082299590110779
T =  301  loss:  29.11808967590332  acc:  0.7087649703025818
T =  301  loss:  29.058895111083984  acc:  0.7093899846076965
T =  301  lo